In [3]:
#https://db.netkeiba.com/race/202005020106/
#レース情報のスクレイピング
import pandas as pd
import numpy as np
import requests
from lxml import html
from bs4 import BeautifulSoup
URL = 'https://db.netkeiba.com/race/202005020102/'
HTML = requests.get(URL).content
COURSE_INFO_X_PATH = '/html/body/div[1]/div[2]/div[1]/div/div/div/diary_snap/div/div/dl/dd/p/diary_snap_cut/span'

def get_elements_around_tabel(soup):
    table_html = soup.find('table')
    elements = table_html.find_all(nowrap="nowrap")
    return elements

def get_texts_from_elements(elements):
    texts = [element.text.replace('\n', '') for element in elements]
    return texts

def reshape_texts_to_df(texts):
#上手い書き方求む
    array = np.array(texts).reshape(int(len(texts)/21), 21)
    df = pd.DataFrame(array[1:], columns = array[0])
    return df

def get_course_info(soup):
    lxml = html.fromstring(str(soup))
    course_info_element = lxml.xpath(COURSE_INFO_X_PATH)
    course_info = course_info_element[0].text.replace('\xa0', '')
    return course_info

def separate_course_info(course_info):
#ハードコーディングになっちゃった
    road_type = course_info[0]
    curve_info = course_info[1]
    course_length = course_info[2:course_info.find('m')]
    weather = course_info[course_info.find('天候')+5]
    road_situation = course_info[course_info.rfind(road_type)+6]
    time = course_info[course_info.find('発走')+5:]
    return road_type, curve_info, weather, road_situation, time

soup = BeautifulSoup(HTML, 'html.parser')
elements = get_elements_around_tabel(soup)
texts = get_texts_from_elements(elements)
df = reshape_texts_to_df(texts)

course_info = get_course_info(soup)
road_type, curve_info, weather, road_situation, time = separate_course_info(course_info)

df['road_type'] = road_type
df['curve_info'] = curve_info
df['weather'] = weather
df['road_situation'] = road_situation
df['time'] = time

In [4]:
df

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,ﾀｲﾑ指数,...,厩舎ｺﾒﾝﾄ,備考,調教師,馬主,賞金(万円),road_type,curve_info,weather,road_situation,time
0,1,4,7,グローサーベア,セ3,56,ルメール,1:26.1,,**,...,,,[東]矢野英一,社台レースホース,510.0,ダ,左,晴,良,10:40
1,2,2,4,カッチョカバロ,牡3,56,津村明秀,1:26.3,1.1/4,**,...,,,[東]牧光二,平田修,200.0,ダ,左,晴,良,10:40
2,3,2,3,ペガサスターボ,牡3,56,横山和生,1:26.3,ハナ,**,...,,,[東]勢司和浩,千明牧場,130.0,ダ,左,晴,良,10:40
3,4,1,1,ノーブルウルフ,牡3,56,横山武史,1:26.9,3.1/2,**,...,,,[東]久保田貴,吉木伸彦,77.0,ダ,左,晴,良,10:40
4,5,6,11,シャークスコーブ,牡3,56,レーン,1:27.2,1.3/4,**,...,,,[東]上原博之,シルクレーシング,51.0,ダ,左,晴,良,10:40
5,6,7,14,タシロ,牡3,56,岩部純二,1:27.2,ハナ,**,...,,,[東]的場均,ミルファーム,,ダ,左,晴,良,10:40
6,7,8,16,マニールリッヒ,牡3,56,石橋脩,1:27.3,3/4,**,...,,,[東]青木孝文,吉田勝己,,ダ,左,晴,良,10:40
7,8,1,2,ロングイ,牡3,56,野中悠太,1:27.6,2,**,...,,,[東]伊藤大士,佐久間拓士,,ダ,左,晴,良,10:40
8,9,3,5,タイキスピネル,牡3,56,ヒューイ,1:27.8,1.1/4,**,...,,,[東]伊藤圭三,大樹ファーム,,ダ,左,晴,良,10:40
9,10,4,8,マロンジェム,牡3,56,田辺裕信,1:28.2,2.1/2,**,...,,,[東]稲垣幸雄,フロンティア,,ダ,左,晴,良,10:40
